## Web Scraping SSCASN BKN

Okay, so here’s the deal. I’m using this notebook to scrape data from the SSCASN BKN website. Why? Because I’m trying to figure out which job formations have a higher chance of getting me through the CPNS 2025 selection process. Instead of manually shifting through pages and pages of information, Why not let Python do the boring work?

This project is super personal and I’m basically building my own cheat sheet to make smarter decisions during the application process. Plus, I get to brush up on my web scraping skills while I’m at it. Win-win, right?

What’s the Plan?

The SSCASN website has everything I need, but it’s not exactly user-friendly for data nerds like me. Especially the filter that only allow us to select one-selection per load. Hence, for the python scrape automation challenge is:

- Dropdown menus that dynamically load options.
- Pagination, because of course, all the good stuff is spread across multiple pages.
- Dynamic tables that make scraping a bit tricky.
- The job description of each formation is described in different link. How to gather all that data needed??

The goal? 

Automate the whole process so I can get clean data in one go. Then let's analyze the data retrieved

In [1]:
import pandas as pd
import math
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [15]:
# This is path to the chromedriver. You can adjust as you need
path_chromedriver = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")

chrome_options = Options()
service = Service(executable_path=path_chromedriver)

In [16]:
# Let's load the Driver
driver = webdriver.Chrome(service=service, options=chrome_options)
url = "https://sscasn.bkn.go.id/" # this is the website links
driver.get(url)

This function was initially created for development purposes to reset the web driver back to the homepage of the SSCASN website. It was particularly useful for troubleshooting or ensuring the code ran smoothly by restarting the scraping process from a clean state. 

Now, it is also used as a convenient way to return to the homepage during operations

In [4]:
def reset_scrap(): 
    """
    This function is created to reset the website into homepage.    
    """
    url = "https://sscasn.bkn.go.id/"
    driver.get(url)  
reset_scrap()

Here I limit the filters into my desired criteria, you may adjust it as you need 
(make sure the text matches exactly what already on the websites)

In [5]:
jenjang = "S-1/Sarjana"
pengadaan = "CPNS"

Since I want to perform multiple searchers, My plan is to loop through the prodi list and retrieve all the data for each prodi.

In [6]:
prodi_list = [
    "S-1 ILMU STATISTIK",
    "S-1 ILMU STATISTIKA",
    "S-1 KEPENDUDUKAN DAN STATISTIK",
    "S-1 KOMPUTASI STATISTIKA",
    "S-1 MATEMATIKA STATISTIKA",
    "S-1 STATISTIK",
    "S-1 STATISTIK (DATA ANALISIS)",
    "S-1 STATISTIK KEUANGAN",
    "S-1 STATISTIK KOMPUTASI",
    "S-1 STATISTIKA",
    "S-1 STATISTIKA BISNIS DAN INDUSTRI",
    "S-1 STATISTIKA DAN SAINS DATA",
    "S-1 STATISTIKA TERAPAN",
    "S-1/D-IV STATISTIK"
]

Here's is the data to interact with the filters.

In [7]:
all_data = [] #initiate base data

In [8]:
def interact_with_filter(jenjang,prodi):
    # Select the "Jenjang Pendidikan"
    jenjang_pendidikan = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="pencarian"]/div/div/form/div[1]/div[1]/div/div/div/input'))
    )
    jenjang_pendidikan.click()

    options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//li[contains(text(), 'S-1/Sarjana')]"))
    )

    for option in options:
        if option.text == jenjang:
            option.click() 
            break
    time.sleep(1)  # Give the website time to load

    # Select the "Prodi"
    prodi_path = '//*[@id="pencarian"]/div/div/form/div[1]/div[2]/div/div/div/input'
    prodi_select = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, prodi_path))
    )
    prodi_select.click()

    options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, f"//li[contains(text(),'{prodi}')]"))
    )

    for option in options:
        if option.text == prodi:
            option.click()
            break

    time.sleep(1)  # Give the website time to load

    # Select the "CPNS"
    pengadaan_path = '//*[@id="pencarian"]/div/div/form/div[1]/div[4]/div/div/div/input'
    pengadaan_select = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, pengadaan_path))
    )
    pengadaan_select.click()

    options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, f"//li[contains(text(),'{pengadaan}')]"))
    )

    for option in options:
        if option.text == pengadaan:
            option.click()

    time.sleep(1)  # Again, give the website time to load 

    # Finally, click the Search button
    cari_path = '//*[@id="pencarian"]/div/div/form/div[1]/div[5]/a'
    driver.find_element(By.XPATH, cari_path).click()

    time.sleep(5)  # Give the website time to load the tables

    # Let's check for how many pages there are
    total_formasi_path = driver.find_element(By.XPATH, '//*[@id="daftarFormasi"]/div[2]/div/div/div[3]/ul/li[1]')

    text_halaman = total_formasi_path.text
    total_formasi = text_halaman.split(': ')[1].split()[0]
    total_page = math.ceil(int(total_formasi) / 10)
    
    return total_page

This function defines how to interact with the pagination, specifically the '>' button. 

Why does the code look like this? After analying the html structure, I discovered that the button dynamically changes based on unique patterns. To handle this, first I create a base format and then loop through with criteria to match each logic.

In [9]:
def klik(index):
    """This function created to click the pagination next button"""
    
    button_xpath = f'//*[@id="daftarFormasi"]/div[2]/div/div/div[3]/ul/li[{index}]/button'
    button = driver.find_element(By.XPATH,button_xpath)
    button.click()

and here it is the main agenda, extracting the data. We will extract the data from dynamic table that changes based on pagination and filters. 

In [10]:
def extract_data():
    table_data = []
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//table"))
    )

    rows = table.find_elements(By.XPATH, ".//tbody/tr")

    for row in rows:

        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = {
            "Jabatan": cells[0].text,
            "Instansi": cells[1].text,
            "Unit Kerja": cells[2].text,
            "Formasi": cells[3].text,
            "(PPPK) Khusus disabilitas? (CPNS) Dapat Diisi Disabilitas?": cells[4].text,
            "Penghasilan (juta)": cells[5].text,
            "Jumlah Kebutuhan": cells[6].text,
            "Jumlah Lulus verifikasi": cells[7].text,
            "Link": cells[8].find_element(By.TAG_NAME, "a").get_attribute('href')
        }

        table_data.append(row_data)
    return table_data

In [11]:
def loop_data(start,total_page):
    """This function is created to loop the extracting progress"""
    page_index = 10  
    current_page = start

    for i in range(current_page,total_page-current_page):
        data = extract_data()
        all_data.extend(data)

        if current_page<=3:
            klik(page_index)

        elif current_page == total_page-3 or current_page == 4:
            page_index = 11
            klik(page_index)

        elif current_page >= total_page-2:
            page_index = 10
            klik(page_index)

        else:
            page_index = 12
            klik(page_index)
        
        current_page += 1
        if i%100==0: # adjust the website load
            time.sleep(15)
        else:
            continue
    reset_scrap()
    time.sleep(5)

In [ ]:
for prodi in prodi_list:
    pages = interact_with_filter(jenjang,prodi)
    loop_data(1,pages)

In [24]:
df = pd.DataFrame(all_data).drop_duplicates()
df.to_excel("Statistika CPNS 2024.xlsx", index=False) 

In [25]:
df

,Jabatan,Instansi,Unit Kerja,Formasi,(PPPK) Khusus disabilitas? (CPNS) Dapat Diisi Disabilitas?,Penghasilan (juta),Jumlah Kebutuhan,Jumlah Lulus verifikasi,Link
0,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Tengah,CPNS Umum,Ya,"2,79 - 9,02",2,550,https://sscasn.bkn.go.id/detailformasi/8a01869...
1,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Timur,CPNS Umum,Ya,"2,79 - 9,02",1,346,https://sscasn.bkn.go.id/detailformasi/8a01869...
2,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Barat,CPNS Umum,Ya,"2,79 - 9,02",3,219,https://sscasn.bkn.go.id/detailformasi/8a01869...
3,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Sulawesi Tenggara,CPNS Umum,Ya,"2,79 - 9,02",2,86,https://sscasn.bkn.go.id/detailformasi/8a01869...
4,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Selatan,CPNS Umum,Ya,"2,79 - 9,02",2,188,https://sscasn.bkn.go.id/detailformasi/8a01869...
...,...,...,...,...,...,...,...,...,...
2194,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CIPOCOK JAY...,CPNS Umum,Ya,"2,49 - 3,09",1,6,https://sscasn.bkn.go.id/detailformasi/d83f10d...
2196,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CURUG | KEL...,CPNS Umum,Ya,"2,49 - 3,09",1,3,https://sscasn.bkn.go.id/detailformasi/881845a...
2197,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,"2,79 - 5",1,61,https://sscasn.bkn.go.id/detailformasi/319bdb6...
2198,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,"2,79 - 5",1,43,https://sscasn.bkn.go.id/detailformasi/39692f7...


In [3]:
def scrape_detail_page(link):
    driver.get(link)
    time.sleep(1)  # Tunggu halaman selesai dimuat

    detail_data = {}
    try:
        detail_data['Jurusan'] = driver.find_element(By.XPATH, '/html/body/div[1]/section/div/div/div[2]/div[3]/p').text
    except:
        detail_data['Jurusan'] = "-"
    try:
        detail_data['Uraian Tugas Jabatan'] = driver.find_element(By.XPATH, '/html/body/div[1]/section/div/div/div[2]/div[4]/p').text
    except:
        detail_data['Uraian Tugas Jabatan'] = "-"
    try:
        detail_data['Keahlian Spesifik yang Diharapkan'] = driver.find_element(By.XPATH, '/html/body/div[1]/section/div/div/div[2]/div[5]/p').text
    except:
        detail_data['Keahlian Spesifik yang Diharapkan'] = "-"
    return detail_data

In [63]:
driver = webdriver.Chrome(service=service, options=chrome_options)

In [8]:
df = pd.read_excel("Statistika CPNS 2024.xlsx")

In [ ]:
# Iterate through each link in the dataframe and scrape the detail page
details_list = []
last_index = 0
for index, row in df.iterrows():
    last_index = index
    link = row['Link']
    detail_data = scrape_detail_page(link)
    if detail_data:
        details_list.append(detail_data)

In [29]:
details_df = pd.DataFrame(details_list)

merged_df = pd.concat([df.reset_index(drop=True), details_df.reset_index(drop=True)], axis=1)

merged_df.to_excel("Statistika CPNS 2024 Detail.xlsx", index=False)


In [30]:
merged_df

,Jabatan,Instansi,Unit Kerja,Formasi,(PPPK) Khusus disabilitas? (CPNS) Dapat Diisi Disabilitas?,Penghasilan (juta),Jumlah Kebutuhan,Jumlah Lulus verifikasi,Link,Jurusan,Uraian Tugas Jabatan,Keahlian Spesifik yang Diharapkan
0,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Tengah,CPNS Umum,Ya,"2,79 - 9,02",2,550.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV MANAJEMEN / D-IV PSIKOLOGI / D-IV ILMU HU...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m..."
1,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Timur,CPNS Umum,Ya,"2,79 - 9,02",1,346.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV MANAJEMEN / D-IV PSIKOLOGI / D-IV ILMU HU...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m..."
2,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Barat,CPNS Umum,Ya,"2,79 - 9,02",3,219.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / D-IV EKONOMI / S-1 EKONOMI / S-1 ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m..."
3,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Sulawesi Tenggara,CPNS Umum,Ya,"2,79 - 9,02",2,86.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / D-IV EKONOMI / S-1 EKONOMI / S-1 ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m..."
4,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Selatan,CPNS Umum,Ya,"2,79 - 9,02",2,188.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / S-1 EKONOMI / S-1 FARMASI / D-IV ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1690,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CIPOCOK JAY...,CPNS Umum,Ya,"2,49 - 3,09",1,6.0,https://sscasn.bkn.go.id/detailformasi/d83f10d...,D-IV PEMBANGUNAN EKONOMI KEWILAYAHAN / D-IV AD...,"Melakukan kegatan pengolahan, pelayanan dan ev...",Mengetahui metode pengadministrasian dokumen m...
1691,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CURUG | KEL...,CPNS Umum,Ya,"2,49 - 3,09",1,3.0,https://sscasn.bkn.go.id/detailformasi/881845a...,D-IV SAINS DATA TERAPAN / S-1 STATISTIKA / S-1...,melakukan kegiatan statistik,keterampilan manajerial Kegiatan Pemerintahan
1692,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,"2,79 - 5",1,61.0,https://sscasn.bkn.go.id/detailformasi/319bdb6...,D-IV ADMINISTRASI NEGARA / D-IV ANIMASI / D-IV...,Melaksanakan tugas melakukan kegiatan yang mel...,Mengelola arsip
1693,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,"2,79 - 5",1,43.0,https://sscasn.bkn.go.id/detailformasi/39692f7...,D-IV ADMINISTRASI NEGARA / D-IV ANIMASI / D-IV...,Melaksanakan tugas yang meliputi kegiatan peng...,Mengelola pengembangan infrastruktur


In [ ]:
# Track down the failed scrap
blank_df = merged_df[merged_df['Jurusan']=='-']
# Iterate through each link in the dataframe and scrape the detail page
extra = []
last_index = 0
for index, row in blank_df.iterrows():
    last_index = index
    link = row['Link']
    detail_data = scrape_detail_page(link)
    if detail_data:
        extra.append(detail_data)

In [49]:
extra_df = pd.DataFrame(extra)

# Ensure the indices match for proper merging
extra_df.index = blank_df.index

# Update the blank_df with the new data
blank_df.update(extra_df)

# Update the merge_df with the new data
merged_df.update(blank_df)

In [70]:
merged_df.to_excel("Statistika CPNS 2024 Detail.xlsx", index=False)

## Deepdown Analysis

Let's restructure the data to make it easier to work with. We'll create a new data frame that separated the "Penghasilan" a.k.a Salary from range into differencse cell as  Min and Max Salary. Also, we will count howmany possible "Jurusan" that able to fill that position

In [73]:
df = pd.read_excel("Statistika CPNS 2024 Detail.xlsx")

In [74]:
# Split the "Penghasilan (juta)" column into "Min Penghasilan" and "Max Penghasilan"
df[['Min Penghasilan', 'Max Penghasilan']] = df['Penghasilan (juta)'].str.replace(',', '.').str.split(' - ', expand=True)

# Convert the new columns to numeric types
df['Min Penghasilan'] = pd.to_numeric(df['Min Penghasilan'])
df['Max Penghasilan'] = pd.to_numeric(df['Max Penghasilan'])

# Drop the original "Penghasilan (juta)" column
df.drop(columns=['Penghasilan (juta)'], inplace=True)

# Display the updated dataframe
df

,Jabatan,Instansi,Unit Kerja,Formasi,(PPPK) Khusus disabilitas? (CPNS) Dapat Diisi Disabilitas?,Jumlah Kebutuhan,Jumlah Lulus verifikasi,Link,Jurusan,Uraian Tugas Jabatan,Keahlian Spesifik yang Diharapkan,Min Penghasilan,Max Penghasilan
0,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Tengah,CPNS Umum,Ya,2,550.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV MANAJEMEN / D-IV PSIKOLOGI / D-IV ILMU HU...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02
1,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Timur,CPNS Umum,Ya,1,346.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV MANAJEMEN / D-IV PSIKOLOGI / D-IV ILMU HU...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02
2,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Barat,CPNS Umum,Ya,3,219.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / D-IV EKONOMI / S-1 EKONOMI / S-1 ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02
3,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Sulawesi Tenggara,CPNS Umum,Ya,2,86.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / D-IV EKONOMI / S-1 EKONOMI / S-1 ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02
4,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Selatan,CPNS Umum,Ya,2,188.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / S-1 EKONOMI / S-1 FARMASI / D-IV ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CIPOCOK JAY...,CPNS Umum,Ya,1,6.0,https://sscasn.bkn.go.id/detailformasi/d83f10d...,D-IV PEMBANGUNAN EKONOMI KEWILAYAHAN / D-IV AD...,"Melakukan kegatan pengolahan, pelayanan dan ev...",Mengetahui metode pengadministrasian dokumen m...,2.49,3.09
1691,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CURUG | KEL...,CPNS Umum,Ya,1,3.0,https://sscasn.bkn.go.id/detailformasi/881845a...,D-IV SAINS DATA TERAPAN / S-1 STATISTIKA / S-1...,melakukan kegiatan statistik,keterampilan manajerial Kegiatan Pemerintahan,2.49,3.09
1692,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,1,61.0,https://sscasn.bkn.go.id/detailformasi/319bdb6...,D-IV ADMINISTRASI NEGARA / D-IV ANIMASI / D-IV...,Melaksanakan tugas melakukan kegiatan yang mel...,Mengelola arsip,2.79,5.00
1693,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,1,43.0,https://sscasn.bkn.go.id/detailformasi/39692f7...,D-IV ADMINISTRASI NEGARA / D-IV ANIMASI / D-IV...,Melaksanakan tugas yang meliputi kegiatan peng...,Mengelola pengembangan infrastruktur,2.79,5.00


In [79]:
df['Count of possible Jurusan'] = df['Jurusan'].apply(lambda x: x.count('/') + 1)
df

,Jabatan,Instansi,Unit Kerja,Formasi,(PPPK) Khusus disabilitas? (CPNS) Dapat Diisi Disabilitas?,Jumlah Kebutuhan,Jumlah Lulus verifikasi,Link,Jurusan,Uraian Tugas Jabatan,Keahlian Spesifik yang Diharapkan,Min Penghasilan,Max Penghasilan,Count of possible Jurusan
0,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Tengah,CPNS Umum,Ya,2,550.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV MANAJEMEN / D-IV PSIKOLOGI / D-IV ILMU HU...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02,99
1,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Jawa Timur,CPNS Umum,Ya,1,346.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV MANAJEMEN / D-IV PSIKOLOGI / D-IV ILMU HU...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02,99
2,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Barat,CPNS Umum,Ya,3,219.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / D-IV EKONOMI / S-1 EKONOMI / S-1 ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02,99
3,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Sulawesi Tenggara,CPNS Umum,Ya,2,86.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / D-IV EKONOMI / S-1 EKONOMI / S-1 ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02,99
4,PENATA KEPENDUDUKAN DAN KELUARGA BERENCANA AHL...,Badan Kependudukan dan Keluarga Berencana Nasi...,Perwakilan BKKBN Provinsi Kalimantan Selatan,CPNS Umum,Ya,2,188.0,https://sscasn.bkn.go.id/detailformasi/8a01869...,D-IV HUKUM / S-1 EKONOMI / S-1 FARMASI / D-IV ...,"Melaksanakan kegiatan penatalaksanaan, penyele...","kemampuan komunikasi, bekerja dengan aturan, m...",2.79,9.02,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CIPOCOK JAY...,CPNS Umum,Ya,1,6.0,https://sscasn.bkn.go.id/detailformasi/d83f10d...,D-IV PEMBANGUNAN EKONOMI KEWILAYAHAN / D-IV AD...,"Melakukan kegatan pengolahan, pelayanan dan ev...",Mengetahui metode pengadministrasian dokumen m...,2.49,3.09,32
1691,PRANATA KEWILAYAHAN,Pemerintah Kota Serang,Pemerintah Kota Serang | KECAMATAN CURUG | KEL...,CPNS Umum,Ya,1,3.0,https://sscasn.bkn.go.id/detailformasi/881845a...,D-IV SAINS DATA TERAPAN / S-1 STATISTIKA / S-1...,melakukan kegiatan statistik,keterampilan manajerial Kegiatan Pemerintahan,2.49,3.09,4
1692,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,1,61.0,https://sscasn.bkn.go.id/detailformasi/319bdb6...,D-IV ADMINISTRASI NEGARA / D-IV ANIMASI / D-IV...,Melaksanakan tugas melakukan kegiatan yang mel...,Mengelola arsip,2.79,5.00,85
1693,ARSIPARIS AHLI PERTAMA,Pemerintah Provinsi Kalimantan Selatan,PEMERINTAH PROVINSI KALIMANTAN SELATAN | DINAS...,CPNS Umum,Ya,1,43.0,https://sscasn.bkn.go.id/detailformasi/39692f7...,D-IV ADMINISTRASI NEGARA / D-IV ANIMASI / D-IV...,Melaksanakan tugas yang meliputi kegiatan peng...,Mengelola pengembangan infrastruktur,2.79,5.00,92
